# Tables

In [1]:
import pandas as pd
import os

In [4]:
path2SP = "/Users/ctoruno/OneDrive - World Justice Project/EU Subnational/EU-S Data/Automated Qualitative Checks/Data"

eu_member_states = [
    "Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia",
    "Finland","France","Germany","Greece","Hungary","Ireland","Italy","Latvia",
    "Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal","Romania","Slovakia",
    "Slovenia","Spain","Sweden"
]


### Table 1: Extracted Data

In [3]:
data_tbl01 = {
    "Country": ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark", "Estonia", "Finland", "France", 
                "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", 
                "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"],
    "Total News Extracted": [46145, 21287, 38118, 37068, 34955, 41415, 11312, 12370, 6647, 64527, 
                             45321, 49504, 18111, 48409, 93858, 5487, 14396, 7894, 10842, 
                             23935, 21434, 29624, 33264, 34874, 10211, 112820, 6417],
    "Date Range": ["Mar 06, 2023 - Mar 07, 2024", "Jun 07, 2023 - Mar 07, 2024", "Jul 07, 2023 - Mar 07, 2024",
                   "Jul 07, 2023 - Mar 07, 2024", "Jul 07, 2023 - Mar 08, 2024", "Jul 07, 2023 - Mar 08, 2024",
                   "Aug 07, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 07, 2023 - Mar 07, 2024", "Jan 08, 2024 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 09, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024",
                   "Aug 08, 2023 - Mar 07, 2024", "Aug 08, 2023 - Mar 07, 2024", "Jan 08, 2024 - Mar 07, 2024"]
}

tbl01 = pd.DataFrame(data_tbl01)
total_row = pd.DataFrame([["Total", sum(data_tbl01["Total News Extracted"]), ""]], columns=tbl01.columns)
tbl01_total = pd.concat([tbl01, total_row], ignore_index=True)

In [4]:
tbl01_total.style.hide(axis="index").format({
    'Total News Extracted': '{:,.0f}'
})

### Table 2: Failed Translation

In [5]:
path2exdata = f"{path2SP}/data-extraction-1/data4translation"
extracted_data = [pd.read_parquet(f"{path2exdata}/{country}_tp.parquet.gzip") for country in eu_member_states]
exdata = pd.concat(extracted_data)

In [24]:
path2trdata = f"{path2SP}/data-extraction-1/ready4class"
files = os.listdir(path2trdata)
translated_data = [pd.read_parquet(f"{path2trdata}/{x}") for x in files]
trdata = pd.concat(translated_data)

In [45]:
tbl02 = (
    exdata.country.value_counts().reset_index().sort_values("country")
    .merge(
        trdata.country.value_counts().reset_index(), 
        how = "left", 
        on  = "country"
    )
    .assign(
        success = lambda df: (1-((df['count_x'] - df['count_y']) / df['count_x']))*100
    )
    .rename(
        columns = {
            "country": "Country",
            "count_x": "Extracted News (n)",
            "count_y": "Translated News (n)",
            "success": "Translated News (%)"
        },
        # inplace = True
    )
)

total_row = pd.DataFrame(
    [[
        "European Union", 
        tbl02["Extracted News (n)"].sum(),
        tbl02["Translated News (n)"].sum(),
        tbl02["Translated News (%)"].mean(),
    ]], 
    columns=tbl02.columns
)
tbl02_total = pd.concat([tbl02, total_row], ignore_index=True)

tbl02_total.style.hide(axis="index").format({
    "Extracted News (n)": "{:,.0f}",
    "Translated News (n)": "{:,.0f}",
    "Translated News (%)": "{:,.1f}"
})

### Table 3: First Stage Classification

In [5]:
path2cldata = f"{path2SP}/data-classification-1/0_compiled"

mutate_dict = {
    "Unrelated"      : 0,
    "Related"        : 1,
    "Justice"        : 1,
    "Governance"     : 1,
    "Rule of Law"    : 1,
    "Skipped article": 2
}

In [6]:
cldata = [pd.read_parquet(f"{path2cldata}/{country}_classified.parquet.gzip") for country in eu_member_states]
cldata_master = pd.concat(cldata)

In [75]:
data = []
for country in eu_member_states:
    file = f"{path2cldata}/{country}_classified.parquet.gzip"
    df = pd.read_parquet(file)
    df["topic_related"] = df["topic_related"].str.strip()
    df["relation"] = df["topic_related"].replace(mutate_dict)

    if country == "Czechia":
        location_column = "location_Czech"
    else:
        location_column = f"location_{country}"

    df["related_within"] = df.apply(lambda row: row[location_column] and row["relation"] == 1, axis=1)

    total   = len(df)
    related = (df.relation.value_counts()[1] / total)*100
    within  = (df.related_within.value_counts()[True] / total)*100
    within_ = (df.related_within.value_counts()[True])
    skipped = (df.relation.value_counts()[2] / total)*100

    dict = {
        "Country": country,
        "Related (%)" : related,
        "Related - Within (%)": within,
        "Related - Within (n)": within_,
        "Unclassified (%)": skipped
    }

    data.append(dict)

In [84]:
tbl03 = pd.DataFrame(data)
total_row = pd.DataFrame(
    [[
        "European Union", 
        tbl02["Related (%)"].mean(),
        tbl02["Related - Within (%)"].mean(),
        tbl02["Related - Within (n)"].sum(),
        tbl02["Unclassified (%)"].mean(),
    ]], 
    columns=tbl03.columns
)
tbl03_total = pd.concat([tbl03, total_row], ignore_index=True)

In [85]:
tbl03_total.style.hide(axis="index").format({
    "Related (%)": "{:,.1f}",
    "Related - Within (%)": "{:,.1f}",
    "Related - Within (n)": "{:,.0f}",
    "Unclassified (%)": "{:,.1f}"
})

### Table 4: Second Stage Classification

In [27]:
tbl04_data = (
    cldata_master.copy()
    .drop_duplicates(subset = "id")
    .loc[:,["country", "pillar_1", "pillar_2", "pillar_3", "pillar_4", "pillar_5", "pillar_6", "pillar_7", "pillar_8"]]
    .groupby("country")
    .agg("mean")
    .apply(lambda x: x*100)
    .reset_index()
    .rename(
        columns = {
            "country" : "Country",
            "pillar_1": "Pillar 1",
            "pillar_2": "Pillar 2",
            "pillar_3": "Pillar 3",
            "pillar_4": "Pillar 4",
            "pillar_5": "Pillar 5",
            "pillar_6": "Pillar 6",
            "pillar_7": "Pillar 7",
            "pillar_8": "Pillar 8",
        }
    )
)
total_row = pd.DataFrame(
    [["European Union"] + tbl04_data.iloc[:, 1:].mean().values.tolist()], 
    columns=tbl04_data.columns
)
tbl04_total = pd.concat([tbl04_data, total_row], ignore_index=True)

In [28]:
tbl04_total.style.hide(axis="index").format(precision=1)